# Modelo otimizado

O código comentado explicando cada etapa está em "basic_model.ipynb"

Nessa etapa focamos em comentar apenas as otimizações realizadas

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore

IMG_HEIGHT = 150
IMG_WIDTH = 150

BATCH_SIZE = 32

TRAIN_DIR = '../processed_data/dataset_1_split/train'
VALIDATION_DIR = '../processed_data/dataset_1_split/validation'


train_datagen = ImageDataGenerator(
    rescale=1./255,             
    rotation_range=40,          
    width_shift_range=0.2,      
    height_shift_range=0.2,     
    shear_range=0.2,            
    zoom_range=0.2,             
    horizontal_flip=True,       
    fill_mode='nearest'         
)


validation_datagen = ImageDataGenerator(
    rescale=1./255
)


train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True               
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False              
)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models # type: ignore

# --- ARQUITETURA DA CNN MELHORADA---
model = models.Sequential(name='CNN_Alimentos_Parte1')

# --- BLOCO 1 ---
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), padding='same', name='conv1_1'))
model.add(layers.MaxPooling2D((2, 2), name='pool1'))

# --- BLOCO 2 ---
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same' ,name='conv2_1'))
model.add(layers.MaxPooling2D((2, 2), name='pool2'))

# --- BLOCO 3 ---
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_1'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_2'))    #Camada extra
model.add(layers.MaxPooling2D((2, 2), name='pool3'))

# --- CLASSIFICADOR (MLP) ---
model.add(layers.Flatten(name='flatten'))

model.add(layers.Dense(256, activation='relu', name='dense1'))

model.add(layers.Dropout(0.5, name='dropout'))

model.add(layers.Dense(16, activation='softmax', name='output'))

# --- FIM DA ARQUITETURA ---
model.summary()

### Implementando Callbacks para o treinamento

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau # type: ignore

# 1. Salva o melhor modelo encontrado com base na menor perda de validação
model_checkpoint = ModelCheckpoint(
    filepath='modelo_otimizado.h5',  # Nome do arquivo para salvar o melhor modelo
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# 2. Para o treinamento se não houver melhora na perda de validação por 5 épocas
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True, # Restaura os pesos da melhor época encontrada
    verbose=1
)

# 3. Reduz a taxa de aprendizado se a perda de validação não melhorar por 5 épocas
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, # Reduz o LR por um fator de 5 (1/5 = 0.2)
    patience=5,
    verbose=1
)

### Compilação e Treinamento

In [ ]:
import tensorflow as tf

model.compile(
    optimizer='adam',                   # Otimizador adam utilizado
    loss='categorical_crossentropy',
    metrics=[                           # Novas métricas utilizadas para avaliação das epocas de treinamento
        'accuracy', 
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

In [ ]:
EPOCHS = 100 # EarlyStopping vai cuidar do resto

history_otimizado = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[model_checkpoint, early_stopping, reduce_lr] # Adiciona a lista de callbacks
)

### Análise do treinamento do modelo

In [ ]:
import matplotlib.pyplot as plt

acc = history_otimizado.history['accuracy']
val_acc = history_otimizado.history['val_accuracy']
loss = history_otimizado.history['loss']
val_loss = history_otimizado.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Acurácia de Treino')
plt.plot(epochs_range, val_acc, label='Acurácia de Validação')
plt.legend(loc='lower right')
plt.title('Acurácia de Treino vs. Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Perda de Treino')
plt.plot(epochs_range, val_loss, label='Perda de Validação')
plt.legend(loc='upper right')
plt.title('Perda de Treino vs. Validação')
plt.xlabel('Épocas')
plt.ylabel('Perda (Loss)')
plt.show()

### Salvando o modelo otimizado

In [ ]:
# Salva o modelo inteiro (arquitetura + pesos)
model.save('modelo_otimizado.h5')